https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [2]:
import sqlite3
import pandas as pd
con = sqlite3.connect(':memory:')
data = pd.read_csv('boston.csv')
data.head()

data.to_sql('boston', con, index=False, if_exists='replace')
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [19]:
pd.read_sql(
    '''
    SELECT 'CRIM' as Колонка, count (CRIM) as count FROM boston 
    where CRIM is null
    UNION
     SELECT 'ZN' as Колонка, count (CRIM) as count FROM boston 
    where CRIM is null
    UNION
     SELECT 'INDUS' as Колонка, count (CRIM) as count FROM boston 
    where CRIM is null
    UNION
     SELECT 'CHAS' as Колонка, count (CRIM) as count FROM boston 
    where CRIM is null
    UNION
     SELECT 'NOX' as Колонка, count (CRIM) as count FROM boston 
    where CRIM is null
    ''',
    con,
)

,Колонка,count
0,CHAS,0
1,CRIM,0
2,INDUS,0
3,NOX,0
4,ZN,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [21]:
pd.read_sql(
    '''
    SELECT 'CRIM' as Колонка, count(DISTINCT CRIM) as unique_count FROM boston 
    UNION
    SELECT 'ZN' as Колонка, count(DISTINCT ZN) as unique_count FROM boston 
    UNION
    SELECT 'INDUS' as Колонка, count(DISTINCT INDUS) as unique_count FROM boston 
    UNION
    SELECT 'CHAS' as Колонка, count(DISTINCT CHAS) as unique_count FROM boston 
    UNION
    SELECT 'NOX' as Колонка, count(DISTINCT NOX) as unique_count FROM boston 
        ''',
    con,
)

,Колонка,unique_count
0,CHAS,2
1,CRIM,504
2,INDUS,76
3,NOX,81
4,ZN,26


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [27]:
pd.read_sql(
    '''
    SELECT 'CRIM' as Nazv FROM boston
    where 
    (SELECT min(CRIM) FROM boston) =
    (SELECT AVG(CRIM)
    FROM (SELECT CRIM
      FROM boston
      ORDER BY CRIM
      LIMIT 2
      OFFSET (SELECT (COUNT(*) - 1) / 2
              FROM boston)))
              
     UNION
     
     SELECT 'ZN' as Nazv FROM boston
    where 
    (SELECT min(ZN) FROM boston) =
    (SELECT AVG(ZN)
    FROM (SELECT ZN
      FROM boston
      ORDER BY ZN
      LIMIT 2
      OFFSET (SELECT (COUNT(*) - 1) / 2
              FROM boston)))
   
    UNION
               
     SELECT 'INDUS' as Nazv FROM boston
    where 
    (SELECT min(INDUS) FROM boston) =
    (SELECT AVG(INDUS)
    FROM (SELECT INDUS
      FROM boston
      ORDER BY INDUS
      LIMIT 2
      OFFSET (SELECT (COUNT(*) - 1) / 2
              FROM boston)))
              
    UNION
               
     SELECT 'CHAS' as Nazv FROM boston
    where 
    (SELECT min(CHAS) FROM boston) =
    (SELECT AVG(CHAS)
    FROM (SELECT CHAS
      FROM boston
      ORDER BY CHAS
      LIMIT 2
      OFFSET (SELECT (COUNT(*) - 1) / 2
              FROM boston)))
              
     UNION
     
     SELECT 'NOX' as Nazv FROM boston
    where 
    (SELECT min(NOX) FROM boston) =
    (SELECT AVG(NOX)
    FROM (SELECT NOX
      FROM boston
      ORDER BY NOX
      LIMIT 2
      OFFSET (SELECT (COUNT(*) - 1) / 2
              FROM boston)))
    ''',
    con,
)

,Nazv
0,CHAS
1,ZN


Вывод: Наибольшее число значений в этих столбца равно минимальному значению. Больше трактов не граничит с рекой.
ZN - жилые земели в основном не разделены на участки площадью более 25 000 кв.футов.


4. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [42]:

pd.read_sql(
    '''
with ranks as (
SELECT LSTAT, MEDV,
row_number() OVER (ORDER BY LSTAT DESC) row_num_by_dept
FROM boston
)

select * from ranks
where MEDV =
( SELECT max(MEDV)
  FROM boston
)
    ''',
    con,
)

,LSTAT,MEDV,row_num_by_dept
0,9.53,50.0,304
1,8.88,50.0,326
2,7.44,50.0,366
3,5.12,50.0,440
4,4.63,50.0,457
5,4.45,50.0,465
6,3.73,50.0,480
7,3.70,50.0,481
8,3.32,50.0,489
9,3.26,50.0,490


В домах с самой дорогой стоимостью живет очень малонаселения с низким статусом

5. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [50]:

pd.read_sql(
    '''
    SELECT 'Не граничит' as 'с рекой', avg(MEDV) as 'средняя стоимость' 
    FROM boston 
    WHERE CHAS = 0
    
    UNION
    
    SELECT 'Граничит' as 'с рекой', avg(MEDV) as 'средняя стоимость' 
    FROM boston 
    WHERE CHAS = 1
    ''',
    con,
)

,с рекой,средняя стоимость
0,Граничит,28.440000
1,Не граничит,22.093843


6. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.


  SELECT avg(CRIM) as 'средняя CRIM'         
    FROM boston 
    where CHAS = 1 

In [9]:

pd.read_sql(
    ''' 
   
   SELECT 'CRIM' as река 
   from
       ( select avg(CRIM) as 'CRIMsr'         
    FROM boston 
    where CHAS = 1)
    where CRIMsr >
    
    (select avg(CRIM) as 'sr'         
    FROM boston)
    
    
    union
    
    SELECT 'ZN' as река 
   from
       ( select avg(ZN) as 'ZNsr'         
    FROM boston 
    where CHAS = 1)
    where ZNsr >
    
    (select avg(ZN) as 'sr'         
    FROM boston)
    
    union
    
       
    SELECT 'INDUS' as река 
   from
       ( select avg(INDUS) as 'INDUSsr'         
    FROM boston 
    where CHAS = 1)
    where INDUSsr >
    
    (select avg(INDUS) as 'sr'         
    FROM boston)
     union
    
       
    SELECT 'NOX' as река 
   from
       ( select avg(NOX) as 'NOXsr'         
    FROM boston 
    where CHAS = 1)
    where NOXsr >
    
    (select avg(NOX) as 'sr'         
    FROM boston)
      
      
    ''',
    con,
)





,река
0,INDUS
1,NOX


Вывод: Около реки концентрация оксидов азота и доля акров неторгового бизнеса на город выше. Если дом граничит с рекой это благоприятино влияет на уровень преступности на душу населения
ZN - доля земель под жилую застройку зонирована под участки площадью более 25 000 кв. футов.

7. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [31]:

pd.read_sql(
    '''
   select 'INDUS 100%' as 'показатель', I100 as 'доля' from (
   
   SELECT
    INDUS AS 'I100'
    FROM boston
    ORDER BY INDUS ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 10 / 10 - 1)
        
     union   
     
        select 'NOX 100%' as 'показатель', N100 as 'NOX 100%' from (
     SELECT
    NOX AS 'N100'
    FROM boston
    ORDER BY NOX ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 10 / 10 - 1)   
        
     union   
     
        select 'INDUS  90%' as 'показатель', I90 as 'доля' from (
     SELECT
    INDUS AS 'I90'
    FROM boston
    ORDER BY INDUS ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 9 / 10 - 1)  
        
         union   
     
        select 'NOX  90%' as 'показатель', N90 as 'NOX 90%' from (
     SELECT
    NOX AS 'N90'
    FROM boston
    ORDER BY NOX ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 9 / 10 - 1)   
        
         union   
     
        select 'INDUS  80%' as 'показатель', I80 as 'INDUS 80%' from (
     SELECT
    INDUS AS 'I80'
    FROM boston
    ORDER BY INDUS ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 8 / 10 - 1)  
        
         union   
     
        select 'NOX  80%' as 'показатель', N80 as 'NOX 80%' from (
     SELECT
    NOX AS 'N80'
    FROM boston
    ORDER BY NOX ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 8 / 10 - 1) 
        
        union
        
        select 'INDUS  70%' as 'показатель', I70 as 'доля' from (
   
   SELECT
    INDUS AS 'I70'
    FROM boston
    ORDER BY INDUS ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 7 / 10 - 1)
        
     union   
     
        select 'NOX  70%' as 'показатель', N70 as 'NOX 70%' from (
     SELECT
    NOX AS 'N70'
    FROM boston
    ORDER BY NOX ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 7 / 10 - 1)   
        
     union  
     
     select 'INDUS  60%' as 'показатель', I60 as 'доля' from (
   
   SELECT
    INDUS AS 'I60'
    FROM boston
    ORDER BY INDUS ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 6 / 10 - 1)
        
     union   
     
        select 'NOX  60%' as 'показатель', N60 as 'NOX 60%' from (
     SELECT
    NOX AS 'N60'
    FROM boston
    ORDER BY NOX ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 6 / 10 - 1)   
        
     union   
     
     select 'INDUS  50%' as 'показатель', I50 as 'доля' from (
   
   SELECT
    INDUS AS 'I50'
    FROM boston
    ORDER BY INDUS ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 5 / 10 - 1)
        
     union   
     
        select 'NOX  50%' as 'показатель', N50 as 'NOX 70%' from (
     SELECT
    NOX AS 'N50'
    FROM boston
    ORDER BY NOX ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 5 / 10 - 1)   
        
     union   
     
     select 'INDUS  40%' as 'показатель', I40 as 'доля' from (
   
   SELECT
    INDUS AS 'I40'
    FROM boston
    ORDER BY INDUS ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 4 / 10 - 1)
        
     union   
     
        select 'NOX  40%' as 'показатель', N40 as 'NOX 40%' from (
     SELECT
    NOX AS 'N40'
    FROM boston
    ORDER BY NOX ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 4 / 10 - 1)   
        
     union   
        
        select 'INDUS  30%' as 'показатель', I30 as 'доля' from (
   
   SELECT
    INDUS AS 'I30'
    FROM boston
    ORDER BY INDUS ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 3 / 10 - 1)
        
     union   
     
        select 'NOX  30%' as 'показатель', N30 as 'NOX 30%' from (
     SELECT
    NOX AS 'N30'
    FROM boston
    ORDER BY NOX ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 3 / 10 - 1)   
        
     union   
     select 'INDUS  20%' as 'показатель', I20 as 'доля' from (
   
   SELECT
    INDUS AS 'I20'
    FROM boston
    ORDER BY INDUS ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 2 / 10 - 1)
        
     union   
     
        select 'NOX  20%' as 'показатель', N20 as 'NOX 20%' from (
     SELECT
    NOX AS 'N20'
    FROM boston
    ORDER BY NOX ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 2 / 10 - 1)   
        
     union   
     select 'INDUS  10%' as 'показатель', I10 as 'доля' from (
   
   SELECT
    INDUS AS 'I10'
    FROM boston
    ORDER BY INDUS ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 1 / 10 - 1)
        
     union   
     
        select 'NOX  10%' as 'показатель', N10 as 'NOX 10%' from (
     SELECT
    NOX AS 'N10'
    FROM boston
    ORDER BY NOX ASC
    LIMIT 1
    OFFSET (SELECT
         COUNT(*)
        FROM boston) * 1 / 10 - 1)   
        
        
        
    ''',
    con,
)

,показатель,доля
0,INDUS 10%,2.890
1,INDUS 20%,4.150
2,INDUS 30%,5.860
3,INDUS 40%,7.070
4,INDUS 50%,9.690
5,INDUS 60%,12.830
6,INDUS 70%,18.100
7,INDUS 80%,18.100
8,INDUS 90%,19.580
9,INDUS 100%,27.740


In [ ]:
Вывод: На показатель концентрация оксидов азота влияет не только доля акров неторгового бизнеса